<a href="https://colab.research.google.com/github/Mewmp55/DADS7202HW02/blob/main/scrape_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/Joeclinton1/google-images-download.git

In [ ]:
import os
import shutil
import copy
import time

import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

from skimage import io
import requests
from google_images_download import google_images_download

import numpy as np
import torch
from torch import nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt

1 - เก็บข้อมูล
ฟังก์ชันที่ใช้ในการเก็บข้อมูล จะไปเสิร์ชหารูปภาพตามคีย์เวิร์ดจากกูเกิ้ลและดาวน์โหลดออกมา จากนั้นรูปภาพที่เสิร์ชหามาได้ก็จะถูกแบ่งเก็บไว้เป็นโฟลเดอร์ train กับ test พร้อมกับแบ่งโฟลเดอร์คลาสให้โดยอัตโนมัติ

พารามิเตอร์
query : เอาไว้ใช้ใส่คีย์เวิร์ดเพื่อใช้ในการหารูป สามารถใส่หลายคีย์เวิร์ดได้โดยการใช้เครื่องหมาย , (comma) คั่น เช่น dog, cat
number : จำนวนรูปภาพที่ต้องการหาแต่ละคีย์เวิร์ด
train_ratio : ใส่ค่าตั้งแต่ 0 ถึง 1 เป็นอัตราส่วนของ training set ต่อจำนวนข้อมูลทั้งหมด

In [ ]:
def collect_data(query, number = 50, train_ratio=0.7) :
  # ลบเว้นวรรค
  query = query.replace(' ','')
  classes = query.split(',')
  
  # ค้นหาและดาวน์โหลดรูปภาพจากกูเกิ้ล
  response = google_images_download.googleimagesdownload()
  arguments = {'keywords' : query, 
              'limit' : number, 
              'silent_mode' : True,
              'format' : 'jpg',
              'output_directory' : 'data'}
  paths = response.download(arguments)

  # สร้างโฟลเดอร์สำหรับแบ่ง training set กับ test set
  if not os.path.isdir('data/train') :
    os.mkdir('data/train')
  if not os.path.isdir('data/test') :
    os.mkdir('data/test')
  for x in classes :
    if not os.path.isdir('data/train/'+x) :
      os.mkdir('data/train/'+x)
  
  # แบ่ง training set กับ test set
  n_train = int(train_ratio*number)
  for x in classes :
    files = os.listdir('data/' + x)
    for i in range(n_train) :
      shutil.move('data/' + x + '/' + files[i], 'data/train/' + x + '/')
    shutil.move('data/' + x, 'data/test/')
  
  print('Complete')

In [ ]:
collect_data('durian', number = 88)